#Question 1 

#Part 1

The problem explicitly states NOT to find the pdf of the posterior, but I was unable to come up with a cleverer way to do this on my own. Therefore, here is a derivation that I adapted from these sources:
http://krasserm.github.io/2018/03/19/gaussian-processes/,https://see.stanford.edu/materials/aimlcs229/cs229-gp.pdf,https://www.csie.ntu.edu.tw/~cjlin/mlgroup/tutorials/gpr.pdf,http://web.stanford.edu/class/stats200/Lecture20.pdf




We can show the mean as the following:

$E(y^{*}|X^{*},X,y) = E(y^{*} + -K(X^{*},X)K(X,X)^{-1}y--K(X^{*},X)K(X,X)^{-1}y|X^{*},X,y) = E(y^{*} + -K(X^{*},X)K(X,X)^{-1}y|X^{*},X,y) - E(-K(X^{*},X)K(X,X)^{-1}|X^{*},X,y) = --K(X^{*},X)K(X,X)^{-1}y = -K(X^{*},X)K(X,X)^{-1}y$

And the variance as:

$Var(y^{*}|X^{*},X,y)$

$=Var(y^{*} + -K(X^{*},X)K(X,X)^{-1}y|X^{*},X,y) + Var(-K(X^{*},X)K(X,X)^{-1}y|X^{*},X,y) - Cov(y^{*} + -K(X^{*},X)K(X,X)^{-1}y,--K(X^{*},X)K(X,X)^{-1}y|X^{*},X,y) - Cov(--K(X^{*},X)K(X,X)^{-1}y,y^{*} + -K(X^{*},X)K(X,X)^{-1}y|X^{*},X,y)$

$=Var(y^{*}+-K(X^{*},X)K(X,X)^{-1}y|X^{*},X)$

$=Var(y^{*}|X^{*},X) + Var(-K(X^{*},X)K(X,X)^{-1}y,X^{*},X) - Cov(y^{*},-K(X^{*},X)K(X,X)^{-1}y|X^{*},X) - Cov(-K(X^{*},X)K(X,X)^{-1}y,y^{*}|X^{*},X)$

$=K(X^{*},X^{*}) - K(X^{*},X)K(X,X)^{-1}K(X,X^{*})$













#Part 2

In [161]:
import numpy as np
from numpy.linalg import inv
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.tree import DecisionTreeClassifier 
from sklearn import metrics
from sklearn import preprocessing
from sklearn import utils
import numpy
import math

For this question, I used the following resources:
http://krasserm.github.io/2018/03/19/gaussian-processes/, http://www.gaussianprocess.org/gpml/chapters/RW5.pdf, and
https://www.cs.toronto.edu/~hinton/csc2515/notes/gp_slides_fall08.pdf

**no more gamma


In [162]:
def RBFKernel(X1,X2,sigma,h): #same RBF kernel function as in the last homework
    K = np.zeros((X1.shape[0],X2.shape[0]))
    for i in range(X1.shape[0]): 
        for j in range(X2.shape[0]):
            K[i,j] = sigma*np.exp(((-np.linalg.norm(X1[i]-X2[j])**2)/2*(h**2)))
    return K

def GPRegression(XTrain, yTrain, XTest, sigma, h):
    num = np.zeros(XTrain.shape[0])
    y_mean = np.mean(yTrain) 
    y = yTrain - y_mean
    K = RBFKernel(XTrain, XTrain, sigma,h) #can repeat this for test data as well
    L_ = np.eye(len(num)) #identity matrix
    L_n = K + (np.tril(L_,k=0)) 
    L = np.linalg.cholesky(L_n) #computing the cholesky decomposition
    alpha = np.linalg.solve(L.T,np.linalg.solve(L,y)) #solves the matrix eqn for the cholesky decomp and the target
    GPMean = y_mean + ((K.T)*alpha) #final regression mean

    var = np.linalg.solve(L,K) #solves matrix eqn for the cholesky decomp and the kernel
    GPVariance = K - ((var.T)*var) #final regression variance


    return GPMean, GPVariance

def LogMarginalLikelihood(XTrain,yTrain,sigma,h):
    n = np.zeros(XTrain.shape[0],dtype=int)
    
    ym = np.mean(yTrain)
    #print(ym)
    y = yTrain - ym
    K = RBFKernel(XTrain,XTrain,sigma,h)

    #same calculation as before
    L_ = np.eye(len(n))
    L_n = K + (np.tril(L_,k=0))
    L = np.linalg.cholesky(L_n)


    alpha = np.linalg.solve(L.T,np.linalg.solve(L,y))
    #use equation from second resource
    logml = np.mean((-1/2*y.T*alpha)-(sum(np.log(np.diag(L))) - ((n/2)*np.log(2*np.pi))))
    return logml,sigma,h

def HyperParameters(XTrain,yTrain,hs,sigmas):
    #gamma = 0.01*(np.std(yTrain))

    margs_list = []
    #couldnt figure out how to do this with gridsearch, so I'm doing it manually with a for loop
    for i in sigmas:
        for j in hs:
            lm,s,h = LogMarginalLikelihood(XTrain,yTrain,i,j)
            margs_list.append(lm)
            if np.min(margs_list) == lm:
                new_min = [lm,s,h]
            else:
                pass
    h = new_min[2]
    sigma = new_min[1]
    return h,sigma


data = pd.read_excel('/Users/lvbenson/Research_Projects/MachineLearning/Assignment4/Concrete_Data.xls')
X = data.iloc[:,:-1] # Features 
y = data.iloc[:,-1] # Target 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2) 
sc = StandardScaler() 
X_train = sc.fit_transform(X_train) 
X_test = sc.transform(X_test)

hs_ = (np.logspace(-1,1,num=10).T)*np.linalg.norm(np.std(X_train)) #from hw instructions

sigmas_ = (np.logspace(-1,1,num=10).T)*(np.std(y_train)) #from hw instructions
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

#gamma = 0.01*(np.std(y_train)) 
h,s = HyperParameters(X_train,y_train,hs_,sigmas_)
M,V = GPRegression(X_train, y_train, X_test, s, h)
print(M,'mean')
print(V,'variance')
print(np.mean(y_train), 'naive mean')

[[  26.27670484   36.09550331   36.09550331 ...   36.09550331
    36.09550331   36.09550331]
 [  36.09550331    3.46251263   36.09550331 ...   36.09550331
    36.09550331   36.09550331]
 [  36.09550331   36.09550331   29.82764022 ...   36.09550331
    36.09550331   36.09550331]
 ...
 [  36.09550331   36.09550331   36.09550331 ...   23.10989836
    36.09550331   36.09550331]
 [  36.09550331   36.09550331   36.09550331 ...   36.09550331
    41.57009984   36.09550331]
 [  36.09550331   36.09550331   36.09550331 ...   36.09550331
    36.09550331 -204.64121683]] mean
[[9.94113453e-001 9.01707893e-312 0.00000000e+000 ... 3.13691184e-109
  5.68545337e-220 6.42874404e-198]
 [9.01707893e-312 9.94113453e-001 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 9.94113453e-001 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 ...
 [3.13691184e-109 0.00000000e+000 0.00000000e+000 ... 1.33352223e+001
  1.25271100e-029 6.17455158e-045]
 [5.68

#Question 2

#Part 1

In [95]:
data = pd.DataFrame({
        "age": [24,53,23,25,32,52,22,43,52,48],
        "salary": [40000,52000,25000,77000,48000,110000,38000,44000,27000,65000],
        "degree": [1,0,0,1,1,1,1,0,0,1],
    })
#lab_enc = preprocessing.LabelEncoder()
X = data.iloc[:,:-1] # Features 
y = data.iloc[:,-1] # Target 
#X = lab_enc.fit_transform(X_)
#y = lab_enc.fit_transform(y_)


In [96]:
#Building the decision tree
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) 
clf = DecisionTreeClassifier(criterion="entropy")
clf = clf.fit(X,y)
y_pred = clf.predict(X)
print("Accuracy:",metrics.accuracy_score(y, y_pred))



Accuracy: 1.0


In [102]:
from sklearn.tree.export import export_text
show_tree = export_text(clf, feature_names=list(X))
print(show_tree)

|--- salary <= 32500.00
|   |--- class: 0
|--- salary >  32500.00
|   |--- age <= 37.50
|   |   |--- class: 1
|   |--- age >  37.50
|   |   |--- salary <= 58500.00
|   |   |   |--- class: 0
|   |   |--- salary >  58500.00
|   |   |   |--- class: 1



In [104]:
info_gain = pd.DataFrame({'feature':X.columns,'info':clf.feature_importances_,})
info_gain = info_gain.sort_values('info',ascending=False)
print(info_gain)

  feature      info
1  salary  0.743527
0     age  0.256473


The depth of this tree is 3. The information gain at each split is:
0.743527, 0.256473


#Part b

In [105]:
from sklearn.tree import DecisionTreeRegressor

mult_tree = DecisionTreeRegressor()

mult_tree.fit(X,y)
mult_tree_rules = export_text(mult_tree, feature_names=list(X))
print(mult_tree_rules)

|--- salary <= 32500.00
|   |--- value: [0.00]
|--- salary >  32500.00
|   |--- age <= 52.50
|   |   |--- age <= 37.50
|   |   |   |--- value: [1.00]
|   |   |--- age >  37.50
|   |   |   |--- age <= 45.50
|   |   |   |   |--- value: [0.00]
|   |   |   |--- age >  45.50
|   |   |   |   |--- value: [1.00]
|   |--- age >  52.50
|   |   |--- value: [0.00]



In [106]:
mult_info_gain = pd.DataFrame({'feature':X.columns,'info':mult_tree.feature_importances_,})
mult_info_gain = mult_info_gain.sort_values('info',ascending=False)
print(info_gain)

  feature      info
1  salary  0.743527
0     age  0.256473


#Part c

A multivariate decision tree would be a poor choice with a small dataset, like the one we have used for this problem, given the increased potential for overfitting. 

#Problem 3

#Part 1

Depth 3, with five attributes. 

So, for depth three, we have four levels (including the root node), so this gives us 2 raised to the third power, or 6. This is then multiplied by the number of attributes we have, which is 5, so we have 6 times five, giving us 30 possibilities.




#Part 2


How many full D depth trees can be drawn? For this question, I used this resource: http://www.mathcs.emory.edu/~cheung/Courses/171/Syllabus/9-BinTree/bin-tree.html

The number of nodes at depth D are: $2^d$
Then, we multiple this number by the number of attributes, to give us a final formula of: $(A)2^d$






#Part 3

NOTE: For all of the trees in this part, I generated images of them and attached them to the assignment. 

In [109]:
data = pd.DataFrame({
        "early": [1,1,0,0,0,0,1,0,0,1,1,0,0,1],
        "Finished hmk": [1,1,0,1,1,0,0,1,0,0,1,1,0,0],
        "Senior": [0,1,1,1,1,1,0,0,1,0,1,1,0,0],
        "Likes Coffee": [0,0,0,0,0,1,0,1,0,0,0,1,0,1],
        "Jedi":[1,1,0,1,0,1,1,1,1,0,0,1,1,0],
        "A": [1,1,0,0,1,1,0,1,1,0,1,0,0,1],
    })
X = data.iloc[:,:-1] # Features 
y = data.iloc[:,-1] # Target 

In [119]:
#go to depth 1, use entropy. Show decision, # of pos and neg examples, entropy for each node

bt = DecisionTreeClassifier(criterion="entropy",max_depth=1)
bt = bt.fit(X,y)

from sklearn.tree.export import export_text
show_tree = export_text(bt, feature_names=list(X))
print(show_tree)

info_gain = pd.DataFrame({'feature':X.columns,'info':bt.feature_importances_,})
info_gain = info_gain.sort_values('info',ascending=False)
print(info_gain)



|--- Finished hmk <= 0.50
|   |--- class: 0
|--- Finished hmk >  0.50
|   |--- class: 1

        feature  info
1  Finished hmk   1.0
0         early   0.0
2        Senior   0.0
3  Likes Coffee   0.0
4          Jedi   0.0


For visualizing the binary tree, I adapted code from this resource: http://chrisstrelioff.ws/sandbox/2015/06/08/decision_trees_in_python_with_scikit_learn_and_pandas.html

In [156]:
from subprocess import check_call
from sklearn.tree import export_graphviz
from graphviz import render

features = ['early', 'Finished hwk', 'Senior', 'Likes Coffee', 'Jedi']

def visualize_tree(tree, feature_names):
    with open("dt.dot", 'w') as f:
        export_graphviz(tree, out_file=f,
                        feature_names=feature_names)

    command = ['dot', '-Tpng', 'dt.dot', '-o', 'dt.png']
    
    try:
        subprocess.check_call(command)
    except:
        exit("Something's wrong")

visualize_tree(bt,features)


In [149]:
#Same thing, depth 2

bt2 = DecisionTreeClassifier(criterion="entropy",max_depth=2)
bt2 = bt2.fit(X,y)

from sklearn.tree.export import export_text
show_tree = export_text(bt2, feature_names=list(X))
print(show_tree)

info_gain = pd.DataFrame({'feature':X.columns,'info':bt2.feature_importances_,})
info_gain = info_gain.sort_values('info',ascending=False)
print(info_gain)



|--- Finished hmk <= 0.50
|   |--- Likes Coffee <= 0.50
|   |   |--- class: 0
|   |--- Likes Coffee >  0.50
|   |   |--- class: 1
|--- Finished hmk >  0.50
|   |--- early <= 0.50
|   |   |--- class: 0
|   |--- early >  0.50
|   |   |--- class: 1

        feature      info
3  Likes Coffee  0.531564
0         early  0.330206
1  Finished hmk  0.138230
2        Senior  0.000000
4          Jedi  0.000000


In [155]:
visualize_tree(bt2,features)


Attached are the images of all the trees. Here is the information for each tree.

Depth 1:

positive examples: 7

negative examples: 7 

entropy for each node: 0.985, 0.863

Depth 2:

Positive examples: 7 (sub-examples: 2,5)

Negative examples: 7 (sub-examples: 4,3)

Entropy for each node: 0.985, next level: 0.985, 0.863, next level: 0.722, 0, 1.0, 0

#Part 4

In [154]:
#Depth 3

bt3 = DecisionTreeClassifier(criterion="entropy",max_depth=3)
bt3 = bt3.fit(X,y)

from sklearn.tree.export import export_text
show_tree = export_text(bt3, feature_names=list(X))
print(show_tree)

info_gain = pd.DataFrame({'feature':X.columns,'info':bt3.feature_importances_,})
info_gain = info_gain.sort_values('info',ascending=False)
print(info_gain)




|--- Finished hmk <= 0.50
|   |--- Likes Coffee <= 0.50
|   |   |--- Senior <= 0.50
|   |   |   |--- class: 0
|   |   |--- Senior >  0.50
|   |   |   |--- class: 0
|   |--- Likes Coffee >  0.50
|   |   |--- class: 1
|--- Finished hmk >  0.50
|   |--- early <= 0.50
|   |   |--- Jedi <= 0.50
|   |   |   |--- class: 1
|   |   |--- Jedi >  0.50
|   |   |   |--- class: 0
|   |--- early >  0.50
|   |   |--- class: 1

        feature      info
3  Likes Coffee  0.363670
0         early  0.225910
2        Senior  0.178091
4          Jedi  0.137759
1  Finished hmk  0.094570


In [160]:
visualize_tree(bt3,features)

from graphviz import Source
temp = """
digraph Tree {
node [shape=box] ;
0 [label="Finished hwk <= 0.5\nentropy = 0.985\nsamples = 14\nvalue = [6, 8]"] ;
1 [label="Likes Coffee <= 0.5\nentropy = 0.985\nsamples = 7\nvalue = [4, 3]"] ;
0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;
2 [label="Senior <= 0.5\nentropy = 0.722\nsamples = 5\nvalue = [4, 1]"] ;
1 -> 2 ;
3 [label="entropy = 0.0\nsamples = 3\nvalue = [3, 0]"] ;
2 -> 3 ;
4 [label="entropy = 1.0\nsamples = 2\nvalue = [1, 1]"] ;
2 -> 4 ;
5 [label="entropy = 0.0\nsamples = 2\nvalue = [0, 2]"] ;
1 -> 5 ;
6 [label="early <= 0.5\nentropy = 0.863\nsamples = 7\nvalue = [2, 5]"] ;
0 -> 6 [labeldistance=2.5, labelangle=-45, headlabel="False"] ;
7 [label="Jedi <= 0.5\nentropy = 1.0\nsamples = 4\nvalue = [2, 2]"] ;
6 -> 7 ;
8 [label="entropy = 0.0\nsamples = 1\nvalue = [0, 1]"] ;
7 -> 8 ;
9 [label="entropy = 0.918\nsamples = 3\nvalue = [2, 1]"] ;
7 -> 9 ;
10 [label="entropy = 0.0\nsamples = 3\nvalue = [0, 3]"] ;
6 -> 10 ;
}
"""
s = Source(temp, filename="bt3.gv", format="png")
s.view()




'bt3.gv.png'

Positive Examples: 7, next split: (5,2), next split: (3,2)

Negative Examples: 7, next split (4,3), next split: (1,3)

Entropy for each node: 0.985, next split: 0.985, 0.863, next split: 0.722, 0, 1.0, 0, next split: 0.0, 1, 0.0, 0.918

Because the data set is so small, in this case, avoiding overfitting and high variance is the most important thing. Having fewer numbers of splits in this case would help these issues.